## 160 Topocalc for sky view factor and shade

[Link to PR](https://github.com/USDA-ARS-NWRC/smrf/pull/160)

Changed number of angles to use for calculation from 16 to 72, which changed net_solar and thermal.

## 165 stoporad to Python

[Link to PR](https://github.com/USDA-ARS-NWRC/smrf/pull/165)

In [ ]:
from common import *

In [ ]:
import json
from collections import namedtuple

SnotelSite = namedtuple('SnotelSites', 'name lon lat')

In [ ]:
snobal_dir = Path.home() / 'shared-cryosphere/iSnobal/output'
water_year = 'wy2018'

coarsen_opts = dict(x=2, y=2, keep_attrs=True)

In [ ]:
client = start_cluster(6, 7)
client

In [ ]:
time=slice("2017-10-01", "2018-07-15")

In [ ]:
snotel_dir = Path.home() / 'shared-cryosphere'
snotel_sites = json.load(
        open(snotel_dir / 'Snotel/snotel_sites_2x2.json', 'r')
    )

butte_snotel = SnotelSite('Butte', snotel_sites['Butte']['lon'], snotel_sites['Butte']['lat'])
schofield_snotel = SnotelSite('Schofield', snotel_sites['Schofield']['lon'], snotel_sites['Schofield']['lat'])
taylor_snotel = SnotelSite('Taylor', snotel_sites['Taylor']['lon'], snotel_sites['Taylor']['lat'])

### AGU results

In [ ]:
wy_snow_agu = xr.open_mfdataset(
    (snobal_dir / 'wy2018_run_1/*/snow.nc').as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=datetime.time(23)).sel(time=time)

AGU_VERSION = 'AGU'

butte_snobal_agu = wy_snow_agu.sel(x=butte_snotel.lon, y=butte_snotel.lat)
schofield_snobal_agu = wy_snow_agu.sel(x=schofield_snotel.lon, y=schofield_snotel.lat)
taylor_snobal_agu = wy_snow_agu.sel(x=taylor_snotel.lon, y=taylor_snotel.lat)

### Latest Run - 16 Sky View Angles

In [ ]:
wy_snow = xr.open_mfdataset(
    (snobal_dir / 'wy2018/runs/*/snow.nc').as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=datetime.time(23)).sel(time=time)

CURRENT = 'SMRF UofU - 16 Angles'

butte_snobal = wy_snow.sel(x=butte_snotel.lon, y=butte_snotel.lat)
schofield_snobal = wy_snow.sel(x=schofield_snotel.lon, y=schofield_snotel.lat)
taylor_snobal = wy_snow.sel(x=taylor_snotel.lon, y=taylor_snotel.lat)

### Run - 72 Sky View Angles

In [ ]:
wy_snow_72 = xr.open_mfdataset(
    (snobal_dir / 'wy2018_UofU/runs/*/snow.nc').as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=datetime.time(23)).sel(time=time)

ANGLES_72 = 'SMRF UofU - 72 Angles'

butte_snobal_72 = wy_snow_72.sel(x=butte_snotel.lon, y=butte_snotel.lat)
schofield_snobal_72 = wy_snow_72.sel(x=schofield_snotel.lon, y=schofield_snotel.lat)
taylor_snobal_72 = wy_snow_72.sel(x=taylor_snotel.lon, y=taylor_snotel.lat)

### Comparison Run

In [ ]:
wy_snow_CMP = xr.open_mfdataset(
    (snobal_dir / 'wy2018_smrf_PR_159/runs/*/snow.nc').as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=datetime.time(23)).sel(time=time)

VERSION_CMP = 'PR 159'

butte_snobal_CMP = wy_snow_CMP.sel(x=butte_snotel.lon, y=butte_snotel.lat)
schofield_snobal_CMP = wy_snow_CMP.sel(x=schofield_snotel.lon, y=schofield_snotel.lat)
taylor_snobal_CMP = wy_snow_CMP.sel(x=taylor_snotel.lon, y=taylor_snotel.lat)

## Compare across SMRF versions
### Snow Depth

In [ ]:
albedo_start = pd.to_datetime('2018-03-15')

def compare_runs(sites, snotel):
    plt.figure(figsize=(6,4), dpi=200)
    for label, site in sites.items():
        site.thickness.coarsen(**coarsen_opts).mean().plot(label=label)
    plt.axvline(x=albedo_start, c='orange', linestyle='--')
    plt.title(snotel.name)
    plt.legend();

In [ ]:
compare_runs(
    {
        VERSION_CMP: butte_snobal_CMP,
        ANGLES_72: butte_snobal_72,
        CURRENT: butte_snobal,
        AGU_VERSION: butte_snobal_agu,
    }, 
    butte_snotel
)

In [ ]:
compare_runs(
    {
        VERSION_CMP: schofield_snobal_CMP,
        ANGLES_72: schofield_snobal_72,
        CURRENT: schofield_snobal,
        AGU_VERSION: schofield_snobal_agu,
    }, 
    schofield_snotel
)

In [ ]:
compare_runs(
    {
        VERSION_CMP: taylor_snobal_CMP,
        ANGLES_72: taylor_snobal_72,
        CURRENT: taylor_snobal,
        AGU_VERSION: taylor_snobal_agu,
    }, 
    taylor_snotel
)

### Depth Difference 

In [ ]:
def mean_difference(a, b, time):
    return (a.thickness - b.thickness.sel(time=time)).coarsen(**coarsen_opts).mean()

def run_difference(a, b, site):
    """
       Difference is: site data a - site data b 
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), dpi=150)
    
    mean_difference(a, b , slice("2017-10-01", "2018-03-14")).plot(ax=ax1)
    ax1.set_title('Before albedo decay')
    
    mean_difference(a, b , slice("2018-03-15", "2018-07-15")).plot(ax=ax2)
    ax2.set_title('Added albedo decay')
    
    for ax in [ax1, ax2]:
        ax.set_ylabel('Snow Depth (m)')
    
    fig.suptitle(site.name + ' - Snow Depth Difference')

In [ ]:
run_difference(butte_snobal, butte_snobal_72, butte_snotel)
run_difference(schofield_snobal, schofield_snobal_72, schofield_snotel)
run_difference(taylor_snobal, taylor_snobal_72, taylor_snotel)

## ERW basin 
### Before Albedo decay

In [ ]:
time=slice("2017-10-01", "2018-03-14")
title = "ERW Basin - Difference {0} to {1}: {time.start} - {time.stop}".format('{0}', '{1}', time=time)

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow_CMP.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_agu.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(VERSION_CMP, AGU_VERSION));

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_CMP.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(CURRENT, VERSION_CMP));

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_agu.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(CURRENT, AGU_VERSION));

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow_72.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_agu.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(ANGLES_72, AGU_VERSION));

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_72.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')
plt.plot(butte_snotel.lon[0], butte_snotel.lat[0], 'o', markersize=6)
plt.plot(schofield_snotel.lon[0], schofield_snotel.lat[0], 'o', markersize=6)
plt.plot(taylor_snotel.lon[0], taylor_snotel.lat[0], 'o', markersize=6)


plt.title(title.format(CURRENT, ANGLES_72));

### Added Albedo Decay 

In [ ]:
time=slice("2018-03-15", "2018-07-15")
title = "ERW Basin - Difference {0} to {1}: {time.start} - {time.stop}".format('{0}', '{1}', time=time)

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow_CMP.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_agu.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(VERSION_CMP, AGU_VERSION));

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_CMP.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(CURRENT, VERSION_CMP));

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)

(
    wy_snow.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_agu.thickness.sel(time=time).mean(dim=['time'])
).plot(cmap='RdBu')

plt.title(title.format(CURRENT, AGU_VERSION));

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(12, 8), dpi=200)

(
    wy_snow_72.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_agu.thickness.sel(time=time).mean(dim=['time'])
).plot(ax=ax1, cmap='RdBu')

plt.title(title.format(ANGLES_72, AGU_VERSION));

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(12, 8), dpi=200)

(
    wy_snow.thickness.sel(time=time).mean(dim=['time']) - 
    wy_snow_72.thickness.sel(time=time).mean(dim=['time'])
).plot(ax=ax1, cmap='RdBu')

plt.title(title.format(CURRENT, ANGLES_72));